# GameVerse Network Design

GameVerse is a major gaming console manufacturer that needs to optimize its distribution network for the holiday season. They have:
- 5 production facilities
- 2 consolidation centers
- 5 regional warehouses
- 2 product lines: Pro Console and Standard Console

The goal is to minimize total transportation costs while meeting demand and respecting capacity constraints.

In [1]:
import pandas as pd
import pulp

In [2]:
# Inbound transportation costs (Factory to Consolidation)
inbound_dict = {'Pro':{'Factory 1':{'Consolidation 1':35, 'Consolidation 2':55} ,
                'Factory 2':{'Consolidation 1':28, 'Consolidation 2':72},
                'Factory 3':{'Consolidation 1':42, 'Consolidation 2':18} ,
                'Factory 4':{'Consolidation 1':75, 'Consolidation 2':16} ,
                'Factory 5':{'Consolidation 1':62, 'Consolidation 2':68}},
                'Standard':{'Factory 1':{'Consolidation 1':32, 'Consolidation 2':58} ,
                'Factory 2':{'Consolidation 1':25, 'Consolidation 2':75},
                'Factory 3':{'Consolidation 1':45, 'Consolidation 2':20} ,
                'Factory 4':{'Consolidation 1':78, 'Consolidation 2':18} ,
                'Factory 5':{'Consolidation 1':18, 'Consolidation 2':22}}
}

In [3]:
# Outbound transportation costs (Consolidation to Warehouse)
outbound_dict = {'Pro':{'Consolidation 1':{'Warehouse 1':18, 'Warehouse 2':28, 'Warehouse 3':32, 'Warehouse 4':45, 'Warehouse 5':48} ,
                'Consolidation 2':{'Warehouse 1':72, 'Warehouse 2':28, 'Warehouse 3':25, 'Warehouse 4':18, 'Warehouse 5':22} } ,
                'Standard':{'Consolidation 1':{'Warehouse 1':15, 'Warehouse 2':32, 'Warehouse 3':35, 'Warehouse 4':48, 'Warehouse 5':52} ,
                'Consolidation 2':{'Warehouse 1':78, 'Warehouse 2':32, 'Warehouse 3':28, 'Warehouse 4':20, 'Warehouse 5':25} } ,
}

In [4]:
# Holiday season demand at each warehouse
demand_dict = {'Pro':{'Warehouse 1':150, 'Warehouse 2':60, 'Warehouse 3':80, 'Warehouse 4':110, 'Warehouse 5':15} ,
                'Standard':{'Warehouse 1':25, 'Warehouse 2':55, 'Warehouse 3':50, 'Warehouse 4':120, 'Warehouse 5':200}
}

In [5]:
# Factory production capacity
capacity_dict = {'Pro':{'Factory 1':160, 'Factory 2': 300, 'Factory 3':95, 'Factory 4':150, 'Factory 5':220} ,
            'Standard':{'Factory 1':200, 'Factory 2': 300, 'Factory 3':80, 'Factory 4':50, 'Factory 5':220} ,
            'Combined':{'Factory 1':200, 'Factory 2': 300, 'Factory 3':110, 'Factory 4':160, 'Factory 5':220}
}

In [6]:
# Create lists of locations and product types
factories = ['Factory 1', 'Factory 2', 'Factory 3', 'Factory 4', 'Factory 5']
consolidations = ['Consolidation 1', 'Consolidation 2']
warehouses = ['Warehouse 1', 'Warehouse 2', 'Warehouse 3', 'Warehouse 4', 'Warehouse 5']
consoles = ['Pro', 'Standard']

In [7]:
# Create optimization model
model = pulp.LpProblem("GameVerse_Network", pulp.LpMinimize)

# Decision variables
# Factory to Consolidation flow
factory_to_cons = pulp.LpVariable.dicts("factory_to_cons",
                                      ((c, f, cc) for c in consoles
                                       for f in factories
                                       for cc in consolidations),
                                      lowBound=0)

# Consolidation to Warehouse flow
cons_to_warehouse = pulp.LpVariable.dicts("cons_to_warehouse",
                                 ((c, cc, w) for c in consoles
                                  for cc in consolidations
                                  for w in warehouses),
                                 lowBound=0)

In [8]:
# Objective function
model += (
    pulp.lpSum([factory_to_cons[c,f,cc] * inbound_dict[c][f][cc]
                for c in consoles
                for f in factories
                for cc in consolidations]) +
    pulp.lpSum([cons_to_warehouse[c,cc,w] * outbound_dict[c][cc][w]
                for c in consoles
                for cc in consolidations
                for w in warehouses])
)

In [9]:
# Constraints

# 1. Flow balance at consolidation centers
for c in consoles:
    for cc in consolidations:
        model += pulp.lpSum([factory_to_cons[c,f,cc] for f in factories]) == \
                pulp.lpSum([cons_to_warehouse[c,cc,w] for w in warehouses])

# 2. Meet demand at warehouses
for c in consoles:
    for w in warehouses:
        model += pulp.lpSum([cons_to_warehouse[c,cc,w] for cc in consolidations]) == demand_dict[c][w]

# 3. Factory capacity constraints
for f in factories:
    model += pulp.lpSum([factory_to_cons[c,f,cc] 
                         for c in consoles
                         for cc in consolidations]) <= capacity_dict['Combined'][f]

In [10]:
# Solve the model
model.solve()
print(f"Status: {pulp.LpStatus[model.status]}")
print(f"Total Cost: ${pulp.value(model.objective):,.2f}")

Status: Optimal
Total Cost: $40,435.00


In [11]:
# Display results
print("\nFactory to Consolidation Flows:")
for c in consoles:
    print(f"\n{c}:")
    for f in factories:
        for cc in consolidations:
            if pulp.value(factory_to_cons[c,f,cc]) > 0:
                print(f"{f} -> {cc}: {pulp.value(factory_to_cons[c,f,cc]):,.0f} units")

print("\nConsolidation to Warehouse Flows:")
for c in consoles:
    print(f"\n{c}:")
    for cc in consolidations:
        for w in warehouses:
            if pulp.value(cons_to_warehouse[c,cc,w]) > 0:
                print(f"{cc} -> {w}: {pulp.value(cons_to_warehouse[c,cc,w]):,.0f} units")


Factory to Consolidation Flows:

Pro:
Factory 1 -> Consolidation 1: 75 units
Factory 2 -> Consolidation 1: 170 units
Factory 3 -> Consolidation 2: 10 units
Factory 4 -> Consolidation 2: 160 units

Standard:
Factory 2 -> Consolidation 1: 130 units
Factory 3 -> Consolidation 2: 100 units
Factory 5 -> Consolidation 2: 220 units

Consolidation to Warehouse Flows:

Pro:
Consolidation 1 -> Warehouse 1: 150 units
Consolidation 1 -> Warehouse 2: 60 units
Consolidation 1 -> Warehouse 3: 35 units
Consolidation 2 -> Warehouse 3: 45 units
Consolidation 2 -> Warehouse 4: 110 units
Consolidation 2 -> Warehouse 5: 15 units

Standard:
Consolidation 1 -> Warehouse 1: 25 units
Consolidation 1 -> Warehouse 2: 55 units
Consolidation 1 -> Warehouse 3: 50 units
Consolidation 2 -> Warehouse 4: 120 units
Consolidation 2 -> Warehouse 5: 200 units
